In [5]:
# ============================================================
# 0) Install (Colab)
# ============================================================
!pip -q install "transformers>=4.35.0" accelerate sentencepiece sacrebleu

import os, re, random, inspect, pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    T5Config,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    TrainerCallback,
)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ============================================================
# 1) TrainingArguments compatibility helper
# ============================================================
def build_seq2seq_args(**kwargs):
    sig = inspect.signature(Seq2SeqTrainingArguments.__init__)
    allowed = set(sig.parameters.keys())

    # evaluation_strategy <-> eval_strategy
    if "evaluation_strategy" in kwargs and "evaluation_strategy" not in allowed and "eval_strategy" in allowed:
        kwargs["eval_strategy"] = kwargs.pop("evaluation_strategy")

    clean = {k: v for k, v in kwargs.items() if k in allowed}
    dropped = [k for k in kwargs.keys() if k not in allowed]
    if dropped:
        print("[Info] Dropped unsupported TrainingArguments:", dropped)

    return Seq2SeqTrainingArguments(**clean)

# ============================================================
# 2) ToTTo parser (NO header tokens in input text)
# ============================================================
def parse_totto_string(input_str):
    if input_str is None or (isinstance(input_str, float) and pd.isna(input_str)):
        input_str = ""
    elif not isinstance(input_str, str):
        input_str = str(input_str)

    page_match = re.search(r'\[PAGE\](.*?)(?=\[SEC\]|\[TEXT\]|\[CELL\]|$)', input_str)
    sec_match  = re.search(r'\[SEC\](.*?)(?=\[TEXT\]|\[CELL\]|$)', input_str)
    text_match = re.search(r'\[TEXT\](.*?)(?=\[CELL\]|$)', input_str)

    page_title    = page_match.group(1).strip() if page_match else ""
    section_title = sec_match.group(1).strip() if sec_match else ""
    free_text     = text_match.group(1).strip() if text_match else ""
    if free_text.lower() == "none":
        free_text = ""

    cell_pattern = re.compile(
        r'\[CELL\](.*?)\[TYPE\].*?\[R_HEAD\](.*?)\[C_HEAD\](.*?)(?=\[CELL\]|$)',
        flags=re.DOTALL
    )

    cells = []
    for m in cell_pattern.finditer(input_str):
        val = (m.group(1) or "").strip()
        row = (m.group(2) or "").strip()
        col = (m.group(3) or "").strip()

        if row == "" or row.lower() == "none":
            row = "None"
        if col == "" or col.lower() == "none":
            col = "None"

        cells.append({"val": val, "row": row, "col": col})

    return {"page": page_title, "section": section_title, "text": free_text, "cells": cells}

# ============================================================
# 3) Header semantic init with CACHE (speed!)
# ============================================================
def prepare_semantic_headers_cached(df, cache_path, base_model_name="t5-base"):
    if os.path.exists(cache_path):
        print(f">>> Loading header cache: {cache_path}")
        with open(cache_path, "rb") as f:
            obj = pickle.load(f)
        return obj["header2id"], obj["init_matrix"]

    print(">>> Scanning headers from data...")
    unique_headers = set(["None"])
    for text in tqdm(df["input"].astype(str).tolist()):
        parsed = parse_totto_string(text)
        for c in parsed["cells"]:
            unique_headers.add(c["row"])
            unique_headers.add(c["col"])

    sorted_headers = ["None"] + sorted([h for h in unique_headers if h != "None"])
    header2id = {h: i for i, h in enumerate(sorted_headers)}

    print(">>> Building Embedding Matrix (first time only)...")
    tok = AutoTokenizer.from_pretrained(base_model_name)
    temp_model = T5ForConditionalGeneration.from_pretrained(base_model_name)
    wte = temp_model.get_input_embeddings()

    init_matrix = torch.zeros((len(sorted_headers), temp_model.config.d_model))
    for i, h in enumerate(tqdm(sorted_headers)):
        if i == 0:
            continue
        ids = tok.encode(h, add_special_tokens=False)
        if not ids:
            continue
        with torch.no_grad():
            vecs = wte(torch.tensor([ids]))
            init_matrix[i] = vecs.mean(dim=1).squeeze(0)

    with open(cache_path, "wb") as f:
        pickle.dump({"header2id": header2id, "init_matrix": init_matrix}, f)
    print(f">>> Saved header cache: {cache_path}")

    return header2id, init_matrix

# ============================================================
# 4) Dataset (ONLY <hl> cells) + labels 안정화 (NaN 방지 핵심)
# ============================================================
class StructureT5Dataset(Dataset):
    def __init__(self, df, tokenizer, header2id, max_len=512, tgt_max_len=128):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.header2id = header2id
        self.max_len = max_len
        self.tgt_max_len = tgt_max_len

        self.hl_start = tokenizer.convert_tokens_to_ids("<hl>")
        self.hl_end   = tokenizer.convert_tokens_to_ids("</hl>")
        if self.hl_start in [-1, None] or self.hl_end in [-1, None]:
            raise ValueError("Tokenizer에 <hl>, </hl>이 등록되지 않았습니다.")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        parsed = parse_totto_string(item["input"])

        context = f"Page: {parsed['page']} | Section: {parsed['section']} | Text: {parsed.get('text','')} "

        cell_texts = []
        map_list = []
        for c in parsed["cells"]:
            val = (c["val"] or "").strip()
            if val == "" or val.lower() == "none":
                continue
            cell_texts.append(f"<hl> {val} </hl>")
            map_list.append((self.header2id.get(c["row"], 0), self.header2id.get(c["col"], 0)))

        if len(cell_texts) == 0:
            cell_texts = ["<hl> None </hl>"]
            map_list = [(0, 0)]

        full_text = context + " ".join(cell_texts)

        tok = self.tokenizer(
            full_text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        input_ids = tok["input_ids"].squeeze(0)
        attention_mask = tok["attention_mask"].squeeze(0)

        row_ids = torch.zeros_like(input_ids)
        col_ids = torch.zeros_like(input_ids)

        cell_idx = 0
        in_hl = False
        ids_list = input_ids.tolist()
        for i, tid in enumerate(ids_list):
            if tid == self.hl_start:
                in_hl = True
                continue
            if tid == self.hl_end:
                in_hl = False
                cell_idx += 1
                continue
            if in_hl and cell_idx < len(map_list):
                row_ids[i] = map_list[cell_idx][0]
                col_ids[i] = map_list[cell_idx][1]

        # ---- labels: 빈/NaN 타겟 방지 ----
        target = item["target"]
        if target is None or (isinstance(target, float) and pd.isna(target)):
            target = "."
        elif not isinstance(target, str):
            target = str(target)
        target = target.strip()
        if len(target) == 0 or target.lower() == "nan":
            target = "."

        try:
            labels = self.tokenizer(
                text_target=target,
                max_length=self.tgt_max_len,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            )["input_ids"].squeeze(0)
        except TypeError:
            labels = self.tokenizer(
                target,
                max_length=self.tgt_max_len,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            )["input_ids"].squeeze(0)

        # 혹시라도 전부 pad면 EOS 한 토큰 강제
        if (labels != self.tokenizer.pad_token_id).sum().item() == 0:
            eos = self.tokenizer.eos_token_id if self.tokenizer.eos_token_id is not None else 1
            labels[0] = eos

        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids.long(),
            "attention_mask": attention_mask.long(),
            "row_ids": row_ids.long(),
            "col_ids": col_ids.long(),
            "labels": labels.long(),
        }

# ============================================================
# 5) Model: word + delta  (NO custom from_pretrained)
# ============================================================
class SafeStructureT5(T5ForConditionalGeneration):
    def __init__(self, config, header_init_matrix=None, max_scale=0.05, delta_l2=5e-4):
        super().__init__(config)
        self.max_scale = float(max_scale)
        self.delta_l2 = float(delta_l2)

        if header_init_matrix is not None:
            self.row_header_embeddings = nn.Embedding.from_pretrained(header_init_matrix, freeze=False)
            self.col_header_embeddings = nn.Embedding.from_pretrained(header_init_matrix, freeze=False)
        else:
            self.row_header_embeddings = nn.Embedding(4096, config.d_model)
            self.col_header_embeddings = nn.Embedding(4096, config.d_model)

        self.row_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        self.col_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        self.delta_ln = nn.LayerNorm(config.d_model)
        self.alpha = nn.Parameter(torch.tensor(0.0))

        nn.init.zeros_(self.row_proj.weight)
        nn.init.zeros_(self.col_proj.weight)

    def forward(self, input_ids=None, attention_mask=None, labels=None, row_ids=None, col_ids=None, **kwargs):
        kwargs.pop("num_items_in_batch", None)

        word_embeds = self.shared(input_ids)

        if row_ids is not None and col_ids is not None:
            row_embeds = self.row_header_embeddings(row_ids)
            col_embeds = self.col_header_embeddings(col_ids)

            row_embeds = row_embeds.masked_fill((row_ids == 0).unsqueeze(-1), 0.0)
            col_embeds = col_embeds.masked_fill((col_ids == 0).unsqueeze(-1), 0.0)

            delta = self.row_proj(row_embeds) + self.col_proj(col_embeds)
            delta = self.delta_ln(delta)

            scale = torch.tanh(self.alpha) * self.max_scale
            inputs_embeds = word_embeds + scale * delta
        else:
            delta = None
            scale = None
            inputs_embeds = word_embeds

        outputs = super().forward(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels,
            **kwargs
        )

        # delta 규제
        if (self.delta_l2 > 0.0) and (delta is not None) and (labels is not None):
            mask = ((row_ids != 0) | (col_ids != 0)).float().unsqueeze(-1)
            if mask.sum() > 0:
                reg = ((scale * delta) ** 2 * mask).sum() / mask.sum()
                outputs.loss = outputs.loss + self.delta_l2 * reg

        return outputs

# ============================================================
# 6) NaN guard callback (NaN 나오면 바로 중단)
# ============================================================
class NanGuardCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        # Trainer가 넘겨주는 loss를 직접 받기는 어려워서,
        # log 이벤트에서 체크하는 게 가장 깔끔함
        return control

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return
        for k, v in logs.items():
            if isinstance(v, float) and (np.isnan(v) or np.isinf(v)):
                raise RuntimeError(f"[NaN/Inf detected] step={state.global_step} {k}={v}")
        return control

# ============================================================
# 7) Paths (EDIT)
# ============================================================
TRAIN_PATH = "/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2/totto_preprocessed_train.json"
VAL_PATH   = "/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2/totto_preprocessed_dev.json"
TEST_PATH  = "/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2/totto_preprocessed_test.json"

OUTPUT_DIR = "/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2/safe_structure_t5_word_delta_BIGVAL"
os.makedirs(OUTPUT_DIR, exist_ok=True)
HEADER_CACHE = os.path.join(OUTPUT_DIR, "header_cache.pkl")

# ============================================================
# 8) Load + Clean (NaN/loss 깨짐 방지)
# ============================================================
print(">>> Loading data...")
train_df = pd.read_json(TRAIN_PATH).dropna(subset=["input", "target"]).copy()
val_df   = pd.read_json(VAL_PATH).dropna(subset=["input", "target"]).copy()

train_df = train_df[train_df["target"].astype(str).str.strip().str.len() > 0].copy()
val_df   = val_df[val_df["target"].astype(str).str.strip().str.len() > 0].copy()

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

print("Train:", len(train_df), "Val:", len(val_df))

# ============================================================
# 9) Tokenizer (+ <hl> tokens)
# ============================================================
tokenizer = AutoTokenizer.from_pretrained("t5-base")
tokenizer.add_special_tokens({"additional_special_tokens": ["<hl>", "</hl>"]})

# ============================================================
# 10) Header cache (큰 데이터면 필수로 캐시!)
# ============================================================
header2id, init_matrix = prepare_semantic_headers_cached(train_df, HEADER_CACHE, base_model_name="t5-base")

# ============================================================
# 11) Dataset
# ============================================================
train_dataset = StructureT5Dataset(train_df, tokenizer, header2id)
val_dataset   = StructureT5Dataset(val_df, tokenizer, header2id)

# ============================================================
# 12) Model build + load base T5 weights safely
# ============================================================
config = T5Config.from_pretrained("t5-base")
model = SafeStructureT5(config, header_init_matrix=init_matrix, max_scale=0.05, delta_l2=5e-4)

base = T5ForConditionalGeneration.from_pretrained("t5-base")
model.load_state_dict(base.state_dict(), strict=False)
model.resize_token_embeddings(len(tokenizer))

# speed options
model.gradient_checkpointing_enable()
model.config.use_cache = False
model.to(device)

# ============================================================
# 13) TrainingArguments (대규모 VAL에 맞게: "epoch 평가" + print 최소화)
# ============================================================
use_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

training_args = build_seq2seq_args(
    output_dir=OUTPUT_DIR,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,

    learning_rate=1e-4,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    weight_decay=0.01,
    max_grad_norm=1.0,
    label_smoothing_factor=0.1,

    num_train_epochs=5,

    # ✅ 느린 eval을 자주 하지 않도록 epoch 단위로!
    evaluation_strategy="epoch",
    save_strategy="epoch",

    # ✅ 프린트/로그 최소화
    logging_strategy="epoch",   # (지원 버전이면) epoch에 한 번만 로그
    # logging_steps=999999,      # logging_strategy 미지원이면 이 줄을 사용

    # ✅ 학습 중에는 생성/metric 계산 안함 (속도 핵심)
    predict_with_generate=False,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    remove_unused_columns=False,
    report_to="none",

    fp16=not use_bf16,
    bf16=use_bf16,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100)

# ============================================================
# 14) Trainer (compute_metrics=None: 학습 중 metrics 계산 제거)
# ============================================================
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=None,              # ✅ 학습 중 metric 계산 제거
    callbacks=[NanGuardCallback()],    # ✅ NaN 나오면 즉시 중단
)

print(">>> Starting Training...")
trainer.train()

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(">>> Training done. Best checkpoint loaded (by eval_loss).")

# ============================================================
# 15) Optional: Test generation (120k는 오래 걸리므로 파일로 저장)
# ============================================================
if os.path.exists(TEST_PATH):
    test_df = pd.read_json(TEST_PATH).dropna(subset=["input"]).copy()
    test_df.reset_index(drop=True, inplace=True)

    test_dataset = StructureT5Dataset(
        # test에 target이 없다면 더미로 만들어도 되는데,
        # 현재 class는 target을 참조하므로 test에도 target 컬럼이 필요합니다.
        # 만약 test json에 target이 없다면 아래처럼 더미를 추가하세요:
        test_df.assign(target="."),
        tokenizer,
        header2id
    )

    # 생성 설정 (필요하면 beam 줄이면 속도↑)
    model.generation_config.num_beams = 4
    model.generation_config.max_length = 128
    model.generation_config.no_repeat_ngram_size = 3

    # 120k 전부 생성은 오래 걸림. 그래도 하려면 아래 실행.
    print(">>> Generating TEST predictions (this can be slow for 120k)...")
    preds = trainer.predict(test_dataset, max_length=128, num_beams=4)
    test_preds = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)
    test_preds = [p.strip() for p in test_preds]

    out_path = os.path.join(OUTPUT_DIR, "test_predictions.txt")
    with open(out_path, "w", encoding="utf-8") as f:
        for p in test_preds:
            f.write(p + "\n")
    print("Saved:", out_path)
else:
    print("[Info] TEST_PATH not found. Skip test generation.")


Device: cuda
>>> Loading data...
Train: 120069 Val: 22293


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

>>> Scanning headers from data...


...

>>> Saved header cache: /content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2/safe_structure_t5_word_delta_BIGVAL/header_cache.pkl
>>> Starting Training...


/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,5.502600,2.439723
2,4.892100,2.405067
3,4.744600,2.376581
4,4.655500,2.351520
5,4.600500,2.357511


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


>>> Training done. Best checkpoint loaded (by eval_loss).
[Info] TEST_PATH not found. Skip test generation.


In [7]:
import os
import torch

# 1. 경로 재설정 (오타 방지)
# 박사님이 설정하신 경로 그대로입니다.
SAVE_PATH = "/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2"

print(f">>> 저장 경로 확인: {SAVE_PATH}")
os.makedirs(SAVE_PATH, exist_ok=True)

# 2. 메모리에 모델이 있는지 확인
try:
    if 'trainer' in locals():
        print("✅ Trainer가 메모리에 있습니다. 모델을 저장합니다...")
        trainer.save_model(SAVE_PATH)
        tokenizer.save_pretrained(SAVE_PATH)
        print(">>> [성공] Trainer를 통해 모델과 토크나이저 저장 완료!")

    elif 'model' in locals() and 'tokenizer' in locals():
        print("✅ Trainer는 없지만 model 변수가 있습니다. 수동 저장합니다...")
        model.save_pretrained(SAVE_PATH)
        tokenizer.save_pretrained(SAVE_PATH)
        print(">>> [성공] model.save_pretrained()로 저장 완료!")

    else:
        print("❌ 메모리에 모델 변수(trainer, model)가 없습니다. 런타임이 초기화된 것 같습니다.")

except Exception as e:
    print(f"❌ 저장 중 에러 발생: {e}")

# 3. 파일이 진짜 생겼는지 눈으로 확인
print("\n>>> 📂 저장된 파일 목록 확인:")
if os.path.exists(SAVE_PATH):
    files = os.listdir(SAVE_PATH)
    for f in files:
        print(f" - {f}")

    if "config.json" in files and "model.safetensors" in files: # 또는 pytorch_model.bin
        print("\n🎉 안심하세요! 모델 파일이 정상적으로 보입니다.")
    else:
        print("\n⚠️ 경고: 핵심 파일(config.json, model.safetensors)이 안 보입니다.")
else:
    print("❌ 폴더가 여전히 비어있습니다.")

>>> 저장 경로 확인: /content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2
✅ Trainer가 메모리에 있습니다. 모델을 저장합니다...
>>> [성공] Trainer를 통해 모델과 토크나이저 저장 완료!

>>> 📂 저장된 파일 목록 확인:
 - totto_data
 - ToTTo_논문.pdf
 - ToTTo Model Architecture IDEA.gdoc
 - totto_preprocessed_dev.json
 - totto_preprocessed_train.json
 - safe_structure_t5_final
 - safe_structure_t5_sampled
 - safe_structure_t5_word_delta_BIGVAL
 - config.json
 - generation_config.json
 - model.safetensors
 - tokenizer_config.json
 - special_tokens_map.json
 - added_tokens.json
 - spiece.model
 - tokenizer.json
 - training_args.bin

🎉 안심하세요! 모델 파일이 정상적으로 보입니다.


In [15]:
from collections import Counter
import numpy as np
import sacrebleu

# =========================================================
# 🛠️ [수정 완료] get_ngrams 함수 (변수명 충돌 해결)
# =========================================================
def get_ngrams(text, max_n=4):
    tokens = text.split()
    ngrams = []
    # k: 현재 n-gram 길이 (1~4)
    for k in range(1, max_n + 1):
        # i: 시작 위치
        for i in range(len(tokens) - k + 1):
            ngrams.append(tuple(tokens[i : i + k]))
    return Counter(ngrams)

# =========================================================
# 점수 계산 (predictions, references는 이미 메모리에 있음)
# =========================================================
print(">>> 점수 계산 재시도...")

if 'predictions' not in locals() or 'references' not in locals():
    print("❌ [오류] predictions 변수가 없습니다. 위쪽의 생성 코드를 먼저 실행했는지 확인해주세요.")
else:
    # 1. BLEU Score
    bleu = sacrebleu.corpus_bleu(predictions, [[r] for r in references])

    # 2. PARENT Score
    parent_scores = []

    # 데이터 개수 맞추기 (혹시 모를 인덱스 에러 방지)
    limit = min(len(predictions), len(references), len(val_df))

    # DataFrame 순회
    for i in range(limit):
        pred = predictions[i]
        ref = references[i]
        row = val_df.iloc[i]

        # 테이블 텍스트 재구성
        parsed = parse_totto_string(row['input'])
        table_text = f"{parsed['page']} {parsed['section']} " + " ".join([c['val'] for c in parsed['cells']])

        # N-gram 추출
        pred_ngrams = get_ngrams(pred)
        ref_ngrams = get_ngrams(ref)
        table_ngrams = get_ngrams(table_text)

        # Precision
        overlap = sum((count for ngram, count in pred_ngrams.items() if ngram in ref_ngrams or ngram in table_ngrams))
        prec = overlap / (sum(pred_ngrams.values()) + 1e-9)

        # Recall
        overlap_ref = sum((count for ngram, count in ref_ngrams.items() if ngram in pred_ngrams))
        rec = overlap_ref / (sum(ref_ngrams.values()) + 1e-9)

        # F1 Score
        f1 = 2 * prec * rec / (prec + rec + 1e-9)
        parent_scores.append(f1)

    parent_f1 = np.mean(parent_scores) * 100

    # ---------------------------------------------------------
    # 최종 결과 출력
    # ---------------------------------------------------------
    print("\n" + "="*40)
    print(f"📊 최종 평가 결과 (Delta Injection 적용됨)")
    print(f"✅ BLEU Score: {bleu.score:.2f}")
    print(f"✅ PARENT F1:  {parent_f1:.2f}")
    print("="*40)

>>> 점수 계산 재시도...

📊 최종 평가 결과 (Delta Injection 적용됨)
✅ BLEU Score: 29.15
✅ PARENT F1:  36.37
